In [1]:
import urllib.request
import urllib.parse
import json
import pandas as pd
import numpy as np

# Base URL and resource ID
base_url = 'https://www.data.qld.gov.au/api/3/action/datastore_search'
resource_id = '2bbef99e-9974-49b9-a316-57402b00609c'

# Define the filter query for the site "Mooloolaba"
filters = {
    "Site": "Mooloolaba"
}

# Define the parameters, including the resource ID and the filters
params = {
    'resource_id': resource_id,
    'limit': 24,  # 24 records for the last 12 hours with 30 min intervals
    'q': json.dumps(filters)  # Convert the filters dictionary to a JSON string
}

# Encode the parameters and create the full URL
url = f"{base_url}?{urllib.parse.urlencode(params)}"

# Initialize the dictionary to store results
result_dict = {}

# Make the request
try:
    fileobj = urllib.request.urlopen(url)
    response = fileobj.read()
    data = json.loads(response)
    
    # Store the records in the dictionary
    for record in data.get('result', {}).get('records', []):
        record_id = record.get('_id')
        result_dict[record_id] = record
    
    df = pd.DataFrame.from_dict(result_dict, orient='index')
    
except urllib.error.HTTPError as e:
    print(f"HTTPError: {e.code} - {e.reason}")
except Exception as e:
    print(f"Error: {e}")


In [3]:
def fetch_data():
    # Base URL and resource ID
    base_url = 'https://www.data.qld.gov.au/api/3/action/datastore_search'
    resource_id = '2bbef99e-9974-49b9-a316-57402b00609c'

    # Define the filter query for the site "Mooloolaba"
    filters = {
        "Site": "Mooloolaba"
    }

    # Define the parameters, including the resource ID and the filters
    params = {
        'resource_id': resource_id,
        'limit': 24,  # 24 records for the last 12 hours with 30 min intervals
        'q': json.dumps(filters)  # Convert the filters dictionary to a JSON string
    }

    # Encode the parameters and create the full URL
    url = f"{base_url}?{urllib.parse.urlencode(params)}"

    # Initialize the dictionary to store results
    result_dict = {}

    # Make the request
    try:
        fileobj = urllib.request.urlopen(url)
        response = fileobj.read()
        data = json.loads(response)
        
        # Store the records in the dictionary
        for record in data.get('result', {}).get('records', []):
            record_id = record.get('_id')
            result_dict[record_id] = record
                
    except urllib.error.HTTPError as e:
        print(f"HTTPError: {e.code} - {e.reason}")
    except Exception as e:
        print(f"Error: {e}")

    return result_dict

In [5]:
result_dict = fetch_data()
result_dict

{757: {'_id': 757,
  'Site': 'Mooloolaba',
  'SiteNumber': '4',
  'Seconds': '1723093200',
  'DateTime': '2024-08-08T15:00:00',
  'Latitude': '-26.56511',
  'Longitude': '153.18469',
  'Hsig': '1.454',
  'Hmax': '2.690',
  'Tp': '7.140',
  'Tz': '4.819',
  'SST': '20.80',
  'Direction': '119.50',
  'Current Speed': '-99.90',
  'Current Direction': '-99.90',
  'rank Site': 0.057308756},
 728: {'_id': 728,
  'Site': 'Mooloolaba',
  'SiteNumber': '4',
  'Seconds': '1723039200',
  'DateTime': '2024-08-08T00:00:00',
  'Latitude': '-26.56525',
  'Longitude': '153.18415',
  'Hsig': '1.112',
  'Hmax': '2.000',
  'Tp': '9.090',
  'Tz': '4.598',
  'SST': '20.85',
  'Direction': '99.80',
  'Current Speed': '-99.90',
  'Current Direction': '-99.90',
  'rank Site': 0.057308756},
 729: {'_id': 729,
  'Site': 'Mooloolaba',
  'SiteNumber': '4',
  'Seconds': '1723041000',
  'DateTime': '2024-08-08T00:30:00',
  'Latitude': '-26.56517',
  'Longitude': '153.18439',
  'Hsig': '1.123',
  'Hmax': '1.880',
  

In [56]:
def preprocess_data(result_dict):
    
    # Conversion to dataframe
    df = pd.DataFrame.from_dict(result_dict, orient='index')

    # Renaming
    df.rename(columns = {
        'DateTime':'datetime',
        'Hmax':'wave_height',
        'Tz':'wave_period',
        'Direction': 'wave_direction'
    }, inplace = True)

    # Timestamp format
    df['datetime'] = pd.to_datetime(df['datetime'])
    # df['DateTime'] = df['DateTime'].dt.strftime('%Y-%m-%d %H:%M:%S')    
    df.set_index(keys = 'datetime', inplace=True)
    df = df.asfreq('30T')

    # Keep only desired variables
    target_vars = ['wave_height', 'wave_period', 'wave_direction']
    df = df[target_vars]

    # Null values
    df = df.replace(-99.9, np.nan)

    return df

In [57]:
data = preprocess_data(result_dict)
data 

,wave_height,wave_period,wave_direction
datetime,,,
2024-08-08 00:00:00,2.000,4.598,99.80
2024-08-08 00:30:00,1.880,4.598,119.50
2024-08-08 01:00:00,1.990,4.598,99.80
2024-08-08 01:30:00,2.270,4.494,113.90
2024-08-08 02:00:00,1.900,4.651,108.30
2024-08-08 02:30:00,2.420,4.494,108.30
2024-08-08 03:00:00,2.000,4.878,122.30
2024-08-08 03:30:00,1.910,4.651,112.50
2024-08-08 04:00:00,1.880,4.598,106.90
